In [26]:
import pandas as pd
import re

df = pd.read_csv('synthetic_conversations.csv')

df2 = pd.read_csv('synthetic_conversations2.csv')

df3 = pd.read_csv('synthetic_conversations3.csv')

In [27]:
df = pd.concat([df, df2], ignore_index=True)

In [28]:
df = pd.concat([df, df3], ignore_index=True)

In [30]:

def process_conversation(text):
    system_pattern = r"System: (.+?)(?=\s*User:|\Z)"
    dialogue_pattern = r"(?s)User: (.+?)\s*AI: (.+?)(?=\s*User:|\Z)"
    
    system_match = re.search(system_pattern, text)
    
    matches = re.findall(dialogue_pattern, text)
    
    if not matches:
        return None
    
    conversation = [{"role": "system", "content": ""}]
  
    if system_match:
        conversation[0]['content'] = system_match.group(1).strip()
    
    # Add user and AI messages
    for user_text, ai_text in matches:
        conversation.append({"role": "user", "content": user_text.strip()})
        conversation.append({"role": "assistant", "content": ai_text.strip()})
    
    return conversation

df['messages'] = df['Converse'].apply(process_conversation)

df_cleaned = df.dropna(subset=['messages'])

df_cleaned.to_csv('messages.csv', index=False)
print(df_cleaned[['messages']].head())


                                            messages
0  [{'role': 'system', 'content': ''}, {'role': '...
1  [{'role': 'system', 'content': ''}, {'role': '...
2  [{'role': 'system', 'content': ''}, {'role': '...
3  [{'role': 'system', 'content': ''}, {'role': '...
4  [{'role': 'system', 'content': ''}, {'role': '...


In [31]:
print(df_cleaned.shape)

(30368, 6)


In [32]:
df_cleaned.head(1)

,Converse,Topic,Subtopic,Style,Greetings,messages
0,User: M'he quedat sense idees per a sopar. Què...,Handling ambiguity,Providing multiple possibilities,Dilemmas,Do not include greetings,"[{'role': 'system', 'content': ''}, {'role': '..."


In [33]:
from huggingface_hub import HfApi, HfFolder, Repository, DatasetCard, DatasetCardData
from datasets import load_dataset, DatasetDict, Dataset
import pandas as pd

In [34]:
hf_dataset = load_dataset("csv", data_files="messages.csv")


Generating train split: 0 examples [00:00, ? examples/s]

In [35]:
def filter_rows(example):
    return example['Style'] not in ["Incomplete prompts (continuation task)"]

hf_dataset = hf_dataset.filter(filter_rows)

Filter:   0%|          | 0/30368 [00:00<?, ? examples/s]

In [36]:
hf_dataset.shape

{'train': (29044, 6)}

In [38]:
def string_to_list(row):
    return eval(row['messages'])

hf_dataset = hf_dataset.map(lambda x: {'messages': string_to_list(x)})


Map:   0%|          | 0/29044 [00:00<?, ? examples/s]

In [39]:
from huggingface_hub import create_repo
repo_url = create_repo("pauhidalgoo/patufet-conversa", repo_type="dataset", private=False)

hf_dataset.push_to_hub("pauhidalgoo/patufet-conversa")

print("Dataset uploaded successfully to Hugging Face!")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/30 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/480 [00:00<?, ?B/s]

Dataset uploaded successfully to Hugging Face!
